In [1]:
import os
from fnmatch import fnmatch
import numpy as np
from PIL import Image
from filtering.filters import Median
from feature_extraction.lbp import Lbp
from sklearn import svm
import sklearn
import time
import pickle

In [ ]:
#Read features of training samples
print("loading features")
train_samples_feats_matrix_LBP = np.load("train_samples_feats_matrix_LBP.npy")
print("features loaded")

In [ ]:
print len (train_samples_feats_matrix_LBP[0])

In [ ]:
#Train svn model from training data
t0 = time.time()
#nu : upper bound of training errors (normal data but lie outside of normalty boundry)
svm_model = svm.OneClassSVM(kernel='rbf', gamma=0.1, nu=0.1)
svm_model.fit(train_samples_feats_matrix_LBP)
print("traing time: ", time.time() - t0)

In [ ]:
#load validation samples feature matrix
valid_samples_feats_matrix = np.load("valid_samples_feats_matrix_LBP.npy")

In [2]:
test_samples_feats_matrix = np.load("test_samples_feats_matrix_LBP.npy")

In [3]:
svm_model = pickle.load( open( "SVM_LBP_[0.2, 0.01]", "rb" ))

In [4]:
#Ground truth
y1 = np.repeat(+1, 6000) #normal
y2 = np.repeat(-1, 4000) #abnormal
y = np.concatenate((y1,y2))

In [ ]:
#Preict labels of training dataset
y_predict_train = svm_model.predict(train_samples_feats_matrix_LBP)
print ("training errors", y_predict_train[y_predict_train == -1].size)

In [5]:
#Predict labels of new data points
y_predict = svm_model.predict(test_samples_feats_matrix)
print ("number of samples detected as outliers", y_predict[y_predict == -1].size)
print ("detected as normal", y_predict[y_predict == +1].size)

('number of samples detected as outliers', 8244)
('detected as normal', 1756)


In [6]:
y_predict_score = svm_model.decision_function(test_samples_feats_matrix)

In [ ]:
fpr,tpr,thresh = sklearn.metrics.roc_curve(y, y_predict_score)
for t, thres in enumerate (thresh) :
  
    f1 = sklearn.metrics.f1_score(y, tmp, average = 'binary')
    auc = sklearn.metrics.roc_auc_score(y, y_predict_score)
    print ("f1: ", f1, " auc: ", auc, "thres: ", thres)

In [7]:
f1_binary = sklearn.metrics.f1_score(y, y_predict, pos_label =-1, average = 'binary')
f1_macro = sklearn.metrics.f1_score(y, y_predict, average = 'macro')
auc = sklearn.metrics.roc_auc_score(y, y_predict_score)
Math_Cof = sklearn.metrics.matthews_corrcoef(y, y_predict) 
tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(y, y_predict).ravel()
print ("f1_binary: ", f1_binary, "f1_macro: ", f1_macro, "MAth_cof: ", Math_Cof , " auc: ", auc)
print("confusion matrix", tn, fp, fn, tp)

('f1_binary: ', 0.55374060764456057, 'f1_macro: ', 0.42462687937668975, 'MAth_cof: ', 0.049571847276067137, ' auc: ', 0.55538258333333335)
('confusion matrix', 3390, 610, 4854, 1146)


In [ ]:
import matplotlib.pyplot as pl
import matplotlib
from skimage import io

fpr,tpr,thresh = sklearn.metrics.roc_curve(y,y_predict_score)

f,ax = pl.subplots(1,1)
ax.plot(fpr,tpr,label="SVM")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC curves")
ax.legend(loc="lower right")
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='square', facecolor='white', alpha=0.5)

textstr = '$F1-binary=%.2f$\n$F1-macro=%.2f$\n$math-corcoeffient=%.2f$\n$AUC=%.2f$'%(0.58, 0.63, 0.28, 0.67)
# place a text box in upper left in axes coords
ax.text(0.95, 0.2, textstr, transform=ax.transAxes, fontsize=14,
    horizontalalignment='right', verticalalignment='bottom', bbox=props)

io.show()

In [ ]:
f.savefig('SVM_LBP.png')